In [ ]:
import os
import pandas as pd
from pathlib import Path

In [ ]:
root = Path(os.path.abspath('')).parent
file = root / 'data' / 'factoryReports.csv'

In [ ]:
df = pd.read_csv(file)
df

In [ ]:
df['Urgency'] = df['Urgency'].replace('low', 'Low')
df = df.drop(columns=['Description'])
df

In [ ]:
from pyvis.network import Network
import json

# Initialize the pyvis Network
net = Network(height="1000px", width="100%", directed=True, notebook=True)

# Define colors for each category to visually separate them
category_colors = {
    'Mechanical Failure': '#A3C1DA',
    'Software Failure': '#AED9A0',
    'Electronic Failure': '#F7B7A3',
    'Leak': '#D5A6E1',
}

# Function to add nodes with wrapped text for the category nodes
def add_node_with_wrapped_text(net, id, label, level, shape='ellipse', color=None):
    # Splitting the text into multiple lines if it's too long
    words = label.split()
    max_chars_per_line = 10
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + word) <= max_chars_per_line:
            current_line += word + " "
        else:
            lines.append(current_line)
            current_line = word + " "
    lines.append(current_line)

    wrapped_label = "\n".join(lines)

    net.add_node(id, label=wrapped_label, shape=shape, level=level, color=color)

# Add nodes and edges for each category
for category in df['Category'].unique():
    df_category = df[df['Category'] == category]
    
    # Add category node with wrapped text
    add_node_with_wrapped_text(net, category, category, level=0, shape='circle', color=category_colors[category])
    
    # Add nodes and edges for urgency, resolution, and cost
    urgencies = df_category['Urgency'].unique()
    resolutions = df_category['Resolution'].unique()
    # costs = df_category['Cost'].astype(str).unique()
    
    for urgency in urgencies:
        net.add_node(urgency, label=urgency, level=1)
        net.add_edge(category, urgency, color=category_colors[category])
    for resolution in resolutions:
        net.add_node(resolution, label=resolution, level=2)
        for urgency in urgencies:
            if urgency in df_category[df_category['Resolution'] == resolution]['Urgency'].values:
                net.add_edge(urgency, resolution, color=category_colors[category])
    # for cost in costs:
    #     net.add_node(cost, label=cost, level=3)
    #     for resolution in resolutions:
    #         if resolution in df_category[df_category['Cost'] == int(cost)]['Resolution'].values:
    #             net.add_edge(resolution, cost, color=category_colors[category])

# Configure the layout using a dictionary and convert it to JSON
options = {
    "nodes": {
        "borderWidth": 2,
        "borderWidthSelected": 4,
        "font": {
            "size": 12,
            "face": "arial",
            "multi": "html"
        },
        "shape": "dot"
    },
    "layout": {
        "hierarchical": {
            "enabled": True,
            "levelSeparation": 150,
            "nodeSpacing": 1000,
            "treeSpacing": 200,
            "direction": "UD",  # Top to Bottom
            "sortMethod": "directed"
        }
    },
    "edges": {
        "arrows": {
            "to": {
                "enabled": True,
                "scaleFactor": 1
            }
        },
        "smooth": {
            "type": "cubicBezier",
            "forceDirection": "vertical",
            "roundness": 0.5
        }
    },
    "interaction": {
        "hover": True,
        "selectConnectedEdges": True,
    }
}

# Convert the options dictionary to a JSON string
options_json = json.dumps(options)
net.set_options(options_json)

# Add custom JavaScript for highlighting related edges and nodes on edge selection
custom_js = """
<script type="text/javascript">
  function highlightPath(edgeId) {
    var connectedNodes = new Set();
    var connectedEdges = new Set();
    var queue = [edgeId];
    var allNodes = network.body.data.nodes.get();
    var allEdges = network.body.data.edges.get();
    
    while (queue.length > 0) {
      var currentEdgeId = queue.pop();
      var currentEdge = allEdges.find(edge => edge.id == currentEdgeId);
      if (!currentEdge) continue;
      
      connectedEdges.add(currentEdgeId);
      connectedNodes.add(currentEdge.from);
      connectedNodes.add(currentEdge.to);
      
      var connectedFromEdges = allEdges.filter(edge => edge.to == currentEdge.from);
      var connectedToEdges = allEdges.filter(edge => edge.from == currentEdge.to);
      
      connectedFromEdges.forEach(edge => {
        if (!connectedEdges.has(edge.id)) {
          queue.push(edge.id);
        }
      });
      
      connectedToEdges.forEach(edge => {
        if (!connectedEdges.has(edge.id)) {
          queue.push(edge.id);
        }
      });
    }
    
    allNodes.forEach(node => {
      node.color = connectedNodes.has(node.id) ? '#FF5733' : '#D3D3D3';
    });
    
    allEdges.forEach(edge => {
      edge.color = connectedEdges.has(edge.id) ? '#FF5733' : '#D3D3D3';
    });
    
    network.body.data.nodes.update(allNodes);
    network.body.data.edges.update(allEdges);
  }
  
  network.on("selectEdge", function(params) {
    highlightPath(params.edges[0]);
  });
</script>
"""

# Add the custom JavaScript to the network's HTML
net.html += custom_js

# Save the graph to an HTML file and display it in the notebook
output_filename = "knowledge_graph.html"
net.show(output_filename)